In [1]:
import sys
sys.path.append('/app')

In [2]:
from indexing.meetings_monitor import MeetingsMonitor
from indexing.worker import IndexingWorker

monitor = MeetingsMonitor()

In [3]:
await monitor.print_queue_status()


=== Queue Status ===
Time: 2024-11-19T17:19:07.499319
Queue Size: 311
Processing: 58
Failed: 90
Seen Users: 0
Active Meetings: 0

=== Ready to Process ===
Meeting 959323cb-74f6-449b-980e-884afe30c7b7 - ready
Meeting 7720a983-27cd-4301-bcc1-75fc2338e052 - ready
Meeting 9fb1b6fc-5ae8-4dd6-b074-ea61c9937d5a - ready
Meeting 724e98b8-7245-4696-82bb-bccd08f4ca30 - ready
Meeting 721cd57c-0e22-4abf-8114-004d30ee091c - ready

=== Delayed for Retry ===
Meeting f5e5934a-41fd-40e3-9024-e2df6f4debaa - retry after 2024-11-19T17:23:50.384404
Meeting ab406a36-dabd-4edb-a327-a87c0ced9f44 - retry after 2024-11-19T17:23:50.393891
Meeting fb2aea72-f75d-4790-99dc-bbc4c7219a31 - retry after 2024-11-19T17:23:50.395309

=== Currently Processing ===
Processing: dd58a030-dee0-4672-8d6b-a891554f8a5f
Processing: ed953dc9-002d-4202-905e-269cc982a3f3
Processing: d0e183f7-7c10-40f9-95b1-ee32ae7aeb93
Processing: 6d0f9083-bd20-4ee3-98f3-666f56670395
Processing: e706dbcc-4bd5-4435-b2e1-67a6044f0e1f
Processing: e388aa6

In [106]:
from asyncio import TaskGroup
from collections import deque

In [107]:
processing_tasks = set()
pending_meetings = deque()

In [108]:
from datetime import datetime

In [109]:
from indexing.redis_keys import RedisKeys

In [110]:
from redis import Redis
import asyncio
import logging


import os
REDIS_HOST = os.getenv('REDIS_HOST', '127.0.0.1')
REDIS_PORT = os.getenv('REDIS_PORT', '6379')


In [111]:
redis = Redis(host=REDIS_HOST, port=REDIS_PORT, db=0)

In [112]:
worker = IndexingWorker(redis)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 29 files:   0%|          | 0/29 [00:00<?, ?it/s]

In [113]:
self = worker

In [69]:

if not pending_meetings:
    now = datetime.now().timestamp()
    next_meetings = self.redis.zrevrangebyscore(
        RedisKeys.INDEXING_QUEUE,
        now, '-inf',
        start=0,
        num=self.max_concurrent * 2
    )
    pending_meetings.extend(
        m.decode() if isinstance(m, bytes) else m 
        for m in next_meetings
    )

In [70]:
meeting_id = pending_meetings.popleft()

In [114]:
meeting_id = '76af3e06-e67d-4d9d-a47d-5d253e613aa2'

In [115]:
await self._process_meeting_safe(meeting_id)


Vexa token: a5021b83bcb24b85957fa27dd367727c
User information retrieved successfully.


In [116]:
await self._process_meeting(meeting_id)

Vexa token: a5021b83bcb24b85957fa27dd367727c
User information retrieved successfully.


In [ ]:

task = asyncio.create_task(self._process_meeting_safe(meeting_id))
processing_tasks.add(task)
task.add_done_callback(processing_tasks.discard)

In [ ]:

# Start new tasks if we're below max_concurrent
while len(processing_tasks) < self.max_concurrent and pending_meetings:
    meeting_id = pending_meetings.popleft()
    
    # Skip if already processing
    if self.redis.sismember(RedisKeys.PROCESSING_SET, meeting_id):
        continue
        
    # Create and track new task
    task = asyncio.create_task(self._process_meeting_safe(meeting_id))
    processing_tasks.add(task)
    task.add_done_callback(processing_tasks.discard)

# Wait a bit if we have no work or are at capacity
if not pending_meetings and not processing_tasks:
    await asyncio.sleep(1)
    continue
    
# Wait for at least one task to complete if we're at capacity
if len(processing_tasks) >= self.max_concurrent:
    await asyncio.wait(
        processing_tasks,
        return_when=asyncio.FIRST_COMPLETED
    )


In [5]:
from vexa import VexaAPI

vexa = VexaAPI()
await vexa.get_user_info()
vexa.get_meetings()

Vexa token: 3ae04e20124d40babc5107e658c666b6


ConnectError: [Errno -2] Name or service not known